![](https://tribune-reloaded.s3.amazonaws.com/media/images/2187156-netflixsocialrectangle-1585549450/2187156-netflixsocialrectangle-1585549450.png)

In [ ]:
import numpy as np 
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
from plotly.subplots import make_subplots
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [ ]:
films = pd.read_csv('../input/netflix-shows/netflix_titles.csv')

# Basic information

In [ ]:
films.head()

In [ ]:
films.info()

In [ ]:
films.isna().sum()

# Preprocessing

**First of all, I would like to make a separate year and month of the premiere date (date_added)**

In [ ]:
films['date_added'] = films['date_added'].fillna('No data')

films['year_added'] = films['date_added'].apply(lambda x: x[-4:len(x)])
films['month_added'] = films['date_added'].apply(lambda x: x.split(' ')[0])

films.head(3)

**Then make the age rating in more understandable form**

There are 7 movies and TV shows without rating, it's not a problem to find out the rating. Mostly used [IMDb](https://www.imdb.com/?ref_=nv_home).

In [ ]:
films.query('rating != rating')

In [ ]:
films.iloc[67, 8] = 'R' 
films.iloc[2359, 8] = 'TV-14'
films.iloc[3660, 8] = 'TV-PG'
films.iloc[3736, 8] = 'R'
films.iloc[3737, 8] = 'R'
films.iloc[3738, 8] = 'R'
films.iloc[4323, 8] = 'PG-13'

**To compare the rating and the age group used information from [prime video](https://www.primevideo.com/help/ref=atv_hp_nd_cnt?nodeId=GFGQU3WYEG6FSJFJ)**

In [ ]:
films['rating'].unique()

In [ ]:
films['age_group'] = films['rating']
MR_age = {'TV-MA': 'Adults',
          'R': 'Adults',
          'PG-13': 'Teens',
          'TV-14': 'Young Adults',
          'TV-PG': 'Older Kids',
          'NR': 'Adults',
          'TV-G': 'Kids',
          'TV-Y': 'Kids',
          'TV-Y7': 'Older Kids',
          'PG': 'Older Kids',
          'G': 'Kids',
          'NC-17': 'Adults',
          'TV-Y7-FV': 'Older Kids',
          'UR': 'Adults'}
films['age_group'] = films['age_group'].map(MR_age)

**Some movies and TV shows have several countries. Therefore, I will classify the films for each country.**

In [ ]:
films['country'] = films['country'].fillna('No data')

countries = []
for i in range(len(films.index)):
    x = films.iloc[i, 5].split(',')
    for j in range(len(x)):
        if x[j].strip() not in countries:
            countries.append(x[j].strip())
        else:
            continue
            
countries.remove('')
countries

In [ ]:
for i in range(len(countries)):
    films[countries[i]] = 0
    for j in range(len(films.index)):
        if countries[i] in films.iloc[j, 5]:
            films[countries[i]][j] = 1
            
films.head(3)

Thus, the analysis will take into account the participation of each country.

**Now I want to know all kinds of genres and classify all the movies and TV shows**

In [ ]:
genres = []
for i in range(len(films.index)):
    x = films.iloc[i, 10].split(',')
    for j in range(len(x)):
        if x[j].strip() not in genres:
            genres.append(x[j].strip())
        else:
            continue

genres

In [ ]:
for i in range(len(genres)):
    films[genres[i]] = 0
    for j in range(len(films.index)):
        if genres[i] in films.iloc[j, 10]:
            films[genres[i]][j] = 1

**In duration leave only integers**

In [ ]:
films['duration'] = films['duration'].map(lambda x: x.split(' ')[0]).astype('int')

In [ ]:
films = films.drop(['listed_in', 'director', 'cast', 'rating', 'date_added', 'show_id', 'country', 'No data'], axis = 1) 
# Drop all columns that will not participate in the analysis

**Let's see the result of preprocessing**

In [ ]:
films.sample(3)

# Word Cloud

**Keywords in the description of movies and TV shows**

In [ ]:
from wordcloud import WordCloud
from PIL import Image
import random

text = str(list(films['description'])).replace(',', '').replace('[', '').replace("'", '').replace(']', '').replace('.', '')

def red_color_func(word, font_size, position, orientation, random_state = None, **kwargs):
    return "hsl(0, 100%%, %f%%)" % random.randint(25, 40)

mask = np.array(Image.open('../input/logo-net/netflix-logo-actuel.jpg'))

plt.rcParams['figure.figsize'] = (20, 20)
wordcloud = WordCloud(background_color = 'white', width = 1400,  height = 1400, max_words = 170, mask = mask).generate(text)

wordcloud.recolor(color_func = red_color_func)

plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis('off')
plt.show()

# Dictribution of TV shows and movies

In [ ]:
fig = px.pie(films['type'].value_counts().reset_index(), values = 'type', names = 'index', width = 600, height = 600)
fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.8, 
                  marker = dict(colors = ['#b20710','#221f1f'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Distribution of <br> TV shows and <br> movies', 
                                      x = 0.5, y = 0.5, font_size = 28, showarrow = False, 
                                      font_family = 'Calibri Black',
                                      font_color = 'black')],
                  showlegend = False)
                  
fig.show()

**TOP-5 countries by count of added movies**

In [ ]:
movies_added = pd.DataFrame(columns = ['country', 'movies'], index = range((films.columns.get_loc('Montenegro') + 1) - 7))
for i in range(8, (films.columns.get_loc('Montenegro') + 1)):
    movies = len(films.query('type == "Movie"').index) - films.query('type == "Movie"').iloc[:, i].value_counts().reset_index().iloc[0, 1]
    movies_added.iloc[i - 7, 0] = films.columns[i]
    movies_added.iloc[i - 7, 1] = movies
    
movies_added = movies_added.sort_values('movies', ascending = False).head(5)
movies_added

In [ ]:
sns.set_style('white')
plt.figure(figsize = (12, 8))
plt.title('TOP-5 countries by count of added movies', size = 22, x = 0.5, y = 1.03)
plt.grid(color = 'gray', linestyle = '-', axis = 'y', alpha = 0.2)
a = sns.barplot(data = movies_added, x = movies_added['country'], y = movies_added['movies'], palette = ['#b20710'])
a.tick_params(labelsize = 12)
plt.ylabel('')
plt.xlabel('')
for i in ['right', 'left', 'top']:
    a.spines[i].set_visible(False)
plt.show()

**Dynamics of adding movies by TOP-5 countries**

In [ ]:
USaddM = films.query('`United States` == 1 & type == "Movie"').groupby('year_added').agg({'United States': 'count'})
UKaddM = films.query('`United Kingdom` == 1 & type == "Movie"').groupby('year_added').agg({'United Kingdom': 'count'})
INDaddM = films.query('India == 1 & type == "Movie"').groupby('year_added').agg({'India': 'count'})
CANaddM = films.query('Canada == 1 & type == "Movie"').groupby('year_added').agg({'Canada': 'count'})
FRaddM = films.query('France == 1 & type == "Movie"').groupby('year_added').agg({'France': 'count'})

WaddM = USaddM.join(UKaddM, how = 'outer')\
.join(INDaddM, how = 'outer')\
.join(CANaddM, how = 'outer')\
.join(FRaddM, how = 'outer').fillna(0).reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( 
    x = WaddM['year_added'], 
    y = WaddM['United States'],
    mode = 'lines + markers',
    name = 'US',
    marker_color = '#c43131',
))
fig.add_trace(go.Scatter( 
    x = WaddM['year_added'], 
    y = WaddM['United Kingdom'],
    mode = 'lines + markers',
    name = 'UK',
    marker_color = '#672669',
))
fig.add_trace(go.Scatter( 
    x = WaddM['year_added'], 
    y = WaddM['India'],
    mode = 'lines + markers',
    name = 'India',
    marker_color = '#b8762a',
))
fig.add_trace(go.Scatter( 
    x = WaddM['year_added'], 
    y = WaddM['Canada'],
    mode = 'lines + markers',
    name = 'Canada',
    marker_color = '#245c16',
))
fig.add_trace(go.Scatter( 
    x = WaddM['year_added'], 
    y = WaddM['France'],
    mode = 'lines + markers',
    name = 'France',
    marker_color = '#147f73',
))

fig.update_layout(plot_bgcolor = 'white', title = 'Dynamics of adding movies by TOP-5 countries', 
                  title_font_family = 'Arial', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'Arial', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'Arial', tickfont_color = '#221f1f', tickfont_size = 12)

fig.show()

**TOP-5 countries by count of added TV shows**

In [ ]:
shows_added = pd.DataFrame(columns = ['country', 'TV shows'], index = range((films.columns.get_loc('Montenegro') + 1) - 7))
for i in range(8, (films.columns.get_loc('Montenegro') + 1)):
    shows = len(films.query('type == "TV Show"').index) - films.query('type == "TV Show"').iloc[:, i].value_counts().reset_index().iloc[0, 1]
    shows_added.iloc[i - 7, 0] = films.columns[i]
    shows_added.iloc[i - 7, 1] = shows
    
shows_added = shows_added.sort_values('TV shows', ascending = False).head(5)
shows_added

In [ ]:
sns.set_style('white')
plt.figure(figsize = (12, 8))
plt.title('TOP-5 countries by count of added TV shows', size = 22, x = 0.5, y = 1.03)
plt.grid(color = 'gray', linestyle = '-', axis = 'y', alpha = 0.2)
a = sns.barplot(data = shows_added, x = shows_added['country'], y = shows_added['TV shows'], palette = ['#b20710'])
a.tick_params(labelsize = 12)
plt.ylabel('')
plt.xlabel('')
for i in ['right', 'left', 'top']:
    a.spines[i].set_visible(False)
plt.show()

**Dynamics of adding TV shows by TOP-5 countries**

In [ ]:
USaddS = films.query('`United States` == 1 & type == "TV Show"').groupby('year_added').agg({'United States': 'count'}).drop('data', axis = 0)
UKaddS = films.query('`United Kingdom` == 1 & type == "TV Show"').groupby('year_added').agg({'United Kingdom': 'count'}).drop('data', axis = 0)
JPaddS = films.query('Japan == 1 & type == "TV Show"').groupby('year_added').agg({'Japan': 'count'}).drop('data', axis = 0)
SKaddS = films.query('`South Korea` == 1 & type == "TV Show"').groupby('year_added').agg({'South Korea': 'count'})
CANaddS = films.query('Canada == 1 & type == "TV Show"').groupby('year_added').agg({'Canada': 'count'})

WaddS = USaddS.join(UKaddS, how = 'outer')\
.join(JPaddS, how = 'outer')\
.join(SKaddS, how = 'outer')\
.join(CANaddS, how = 'outer').fillna(0).reset_index()

WaddS.tail(5)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter( 
    x = WaddS['year_added'], 
    y = WaddS['United States'],
    mode = 'lines + markers',
    name = 'US',
    marker_color = '#c43131',
))
fig.add_trace(go.Scatter( 
    x = WaddS['year_added'], 
    y = WaddS['United Kingdom'],
    mode = 'lines + markers',
    name = 'UK',
    marker_color = '#672669',
))
fig.add_trace(go.Scatter( 
    x = WaddS['year_added'], 
    y = WaddS['Japan'],
    mode = 'lines + markers',
    name = 'Japan',
    marker_color = '#b8762a',
))
fig.add_trace(go.Scatter( 
    x = WaddS['year_added'], 
    y = WaddS['South Korea'],
    mode = 'lines + markers',
    name = 'South Korea',
    marker_color = '#245c16',
))
fig.add_trace(go.Scatter( 
    x = WaddS['year_added'], 
    y = WaddS['Canada'],
    mode = 'lines + markers',
    name = 'Canada',
    marker_color = '#147f73',
))

fig.update_layout(plot_bgcolor = 'white', title = 'Dynamics of adding TV shows by TOP-5 countries', 
                  title_font_family = 'Arial', title_font_color = '#221f1f', title_font_size = 20, title_x = 0.5)
fig.update_xaxes(showline = True, linecolor = '#f5f2f2', linewidth = 2, tickfont_family = 'Arial', tickfont_color = '#221f1f', tickfont_size = 12)
fig.update_yaxes(showline = True, linecolor = '#f5f2f2', 
                 showgrid = True, gridwidth = 1, gridcolor = '#f5f2f2',
                 linewidth = 2, tickfont_family = 'Arial', tickfont_color = '#221f1f', tickfont_size = 12)

fig.show()

# Distribution adding content by month

In [ ]:
content_by_month = films.query('month_added != "No" & month_added != ""')\
.groupby('month_added').agg({'type': 'count'}).reset_index()\
.rename(columns = {'type': 'content'}).sort_values('content', ascending = False)

fig = px.pie(labels = content_by_month['month_added'], values = content_by_month['content'], names = content_by_month['month_added'],
            width = 550, height = 550)

colors = ['#6e1919', '#781c1c', '#821f1f', '#871f1f', '#8c2020', '#962121', '#9e2121', '#ab2020', '#b31e1e', '#bf1f1f', '#c91c1c', '#d61818']
fig.update_traces(hoverinfo = 'label + percent', textfont_size = 10,
                  textinfo = 'label + percent', pull = [0.065, 0.06, 0.055, 0.05, 0.045, 0.04, 0.035, 0.03, 0.025, 0.02, 0.015, 0.015],
                  marker = dict(colors = colors[::-1], line = dict(color = 'white', width = 1)))
fig.update_layout(showlegend = False)

fig.show()

New content is most often added in the early winter, and least often added in the summer.

# Duration of movies and TV Shows

In [ ]:
seasons = films.query("type == 'TV Show'").groupby('duration').agg({'title': 'count'}).reset_index()

fig = plt.figure(figsize = (14, 6))

plt.subplot(121)
sns.set_style("white")
plt.title('Movies', size = 20)
a = sns.kdeplot(films.query('type == "Movie"')['duration'], color = '#8c2020', shade = True, alpha = 1)
plt.grid(color = 'gray', linestyle = ':', axis = 'x', zorder = 0,  dashes = (1,7))
for i in ['right', 'left', 'top']:
    a.spines[i].set_visible(False)
a.tick_params(labelsize = 11)
plt.ylabel('')
plt.xlabel('Minutes', size = 14)
plt.yticks([])

plt.subplot(122)
plt.title('TV Shows seasons', size = 20)
plt.grid(color = 'gray', linestyle = '-', axis = 'y', alpha = 0.2)
b = sns.barplot(data = seasons, x = seasons['duration'], y = seasons['title'], palette = ['#8c2020'])
b.tick_params(labelsize = 11)
plt.ylabel('')
plt.xlabel('Seasons', size = 14)
for i in ['right', 'left', 'top']:
    b.spines[i].set_visible(False)

plt.show()

# What age groups are content created for?

In [ ]:
movie_age_groups = films.query('type == "Movie"').groupby('age_group').agg({'age_group': 'count'}).rename(columns = {'age_group': 'content'}).reset_index()
tvshow_age_groups = films.query('type == "TV Show"').groupby('age_group').agg({'age_group': 'count'}).rename(columns = {'age_group': 'content'}).reset_index()

fig = px.pie(labels = movie_age_groups['age_group'], values = movie_age_groups['content'], names = movie_age_groups['age_group'], width = 550, height = 550)

fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.75, 
                  marker = dict(colors = ['#A10115', '#D72C16', '#F0EFEA', '#C0B2B5', '#221f1f'], 
                                line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Movies', 
                                      x = 0.5, y = 0.5, font_size = 32, showarrow = False, 
                                      font_family = 'Calibri Black',
                                      font_color = 'black')],
                  showlegend = False)

fig.show()

####################################################

fig2 = px.pie(labels = tvshow_age_groups['age_group'], values = tvshow_age_groups['content'], names = tvshow_age_groups['age_group'], width = 550, height = 550)

fig2.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label', 
                  hole = 0.75, 
                  marker = dict(colors = ['#A10115', '#D72C16', '#F0EFEA', '#C0B2B5', '#221f1f'], 
                                line = dict(color = 'white', width = 2)))

fig2.update_layout(annotations = [dict(text = 'TV Shows', 
                                      x = 0.5, y = 0.5, font_size = 32, showarrow = False, 
                                      font_family = 'Calibri Black',
                                      font_color = 'black')],
                  showlegend = False)

fig2.show()

# Distribution of the main genres

In [ ]:
films_genres = pd.DataFrame(columns = ['genre', 'count'], index = range(7))
films_genres.iloc[0, 0] = 'Dramas'
films_genres.iloc[0, 1] = films.query('Dramas == 1').agg({'title': 'count'})[0]
films_genres.iloc[1, 0] = 'Comedies'
films_genres.iloc[1, 1] = films.query('Comedies == 1').agg({'title': 'count'})[0]
films_genres.iloc[2, 0] = 'Action & Adventure'
films_genres.iloc[2, 1] = films.query('`Action & Adventure` == 1').agg({'title': 'count'})[0]
films_genres.iloc[3, 0] = 'Documentaries'
films_genres.iloc[3, 1] = films.query('Documentaries == 1').agg({'title': 'count'})[0]
films_genres.iloc[4, 0] = 'Thrillers'
films_genres.iloc[4, 1] = films.query('Thrillers == 1').agg({'title': 'count'})[0]
films_genres.iloc[5, 0] = 'Horror Movies'
films_genres.iloc[5, 1] = films.query('`Horror Movies` == 1').agg({'title': 'count'})[0]
films_genres.iloc[6, 0] = 'Sci-Fi & Fantasy'
films_genres.iloc[6, 1] = films.query('`Sci-Fi & Fantasy` == 1').agg({'title': 'count'})[0]

In [ ]:
fig = px.pie(labels = films_genres['genre'], values = films_genres['count'], names = films_genres['genre'], width = 550, height = 550)
fig.update_traces(textposition = 'inside', 
                  textinfo = 'percent + label',  
                  marker = dict(colors = ['#A10115', '#D72C16', '#F0EFEA', '#C0B2B5', '#221f1f', '#EAD39C', '#7D5E3C'], line = dict(color = 'white', width = 2)))

fig.update_layout(annotations = [dict(text = 'Distribution of genres', 
                                      x = 0.5, y = 1.11, font_size = 26, showarrow = False, 
                                      font_family = 'Calibri Black',
                                      font_color = 'black')],
                  showlegend = False)
                  
fig.show()

# Distribution of main genres in US, India and Japan

In [ ]:
dist = pd.DataFrame(columns = ['country', 'genre', 'count'], index = range(21))
dist.iloc[0:7, 0] = 'US'
dist.iloc[7:14, 0] = 'India'
dist.iloc[14:22, 0] = 'Japan'
dist.iloc[[0, 7, 14], 1] = 'Dramas'
dist.iloc[[1, 8, 15], 1] = 'Comedies'
dist.iloc[[2, 9, 16], 1] = 'Action & Adventure'
dist.iloc[[3, 10, 17], 1] = 'Documentaries'
dist.iloc[[4, 11, 18], 1] = 'Thrillers'
dist.iloc[[5, 12, 19], 1] = 'Horror Movies'
dist.iloc[[6, 13, 20], 1] = 'Sci-Fi & Fantasy'


dist.iloc[0, 2] = films.query('`United States` == 1 & Dramas == 1').agg({'title': 'count'})[0]
dist.iloc[1, 2] = films.query('`United States` == 1 & Comedies == 1').agg({'title': 'count'})[0]
dist.iloc[2, 2] = films.query('`United States` == 1 & `Action & Adventure` == 1').agg({'title': 'count'})[0]
dist.iloc[3, 2] = films.query('`United States` == 1 & Documentaries == 1').agg({'title': 'count'})[0]
dist.iloc[4, 2] = films.query('`United States` == 1 & Thrillers == 1').agg({'title': 'count'})[0]
dist.iloc[5, 2] = films.query('`United States` == 1 & `Horror Movies` == 1').agg({'title': 'count'})[0]
dist.iloc[6, 2] = films.query('`United States` == 1 & `Sci-Fi & Fantasy` == 1').agg({'title': 'count'})[0]

dist.iloc[7, 2] = films.query('India == 1 & Dramas == 1').agg({'title': 'count'})[0]
dist.iloc[8, 2] = films.query('India == 1 & Comedies == 1').agg({'title': 'count'})[0]
dist.iloc[9, 2] = films.query('India == 1 & `Action & Adventure` == 1').agg({'title': 'count'})[0]
dist.iloc[10, 2] = films.query('India == 1 & Documentaries == 1').agg({'title': 'count'})[0]
dist.iloc[11, 2] = films.query('India == 1 & Thrillers == 1').agg({'title': 'count'})[0]
dist.iloc[12, 2] = films.query('India == 1 & `Horror Movies` == 1').agg({'title': 'count'})[0]
dist.iloc[13, 2] = films.query('India == 1 & `Sci-Fi & Fantasy` == 1').agg({'title': 'count'})[0]

dist.iloc[14, 2] = films.query('Japan == 1 & Dramas == 1').agg({'title': 'count'})[0]
dist.iloc[15, 2] = films.query('Japan == 1 & Comedies == 1').agg({'title': 'count'})[0]
dist.iloc[16, 2] = films.query('Japan == 1 & `Action & Adventure` == 1').agg({'title': 'count'})[0]
dist.iloc[17, 2] = films.query('Japan == 1 & Documentaries == 1').agg({'title': 'count'})[0]
dist.iloc[18, 2] = films.query('Japan == 1 & Thrillers == 1').agg({'title': 'count'})[0]
dist.iloc[19, 2] = films.query('Japan == 1 & `Horror Movies` == 1').agg({'title': 'count'})[0]
dist.iloc[20, 2] = films.query('Japan == 1 & `Sci-Fi & Fantasy` == 1').agg({'title': 'count'})[0]

# Because of the classification done for a more accurate analysis, it is impossible to make this table through groupby (which would be many times faster), 
# so I had to write directly :(

In [ ]:
fig = px.sunburst(dist, path = ['country', 'genre'], values = 'count', color = 'country',
                 color_discrete_map = {'US': '#8C2020', 'India': '#EAD39C', 'Japan': '#7D5E3C'},
                 width = 650, height = 650)

fig.update_layout(annotations = [dict(text = 'Distribution of main genres in US, India and Japan', 
                                      x = 0.5, y = 1.11, font_size = 22, showarrow = False, 
                                      font_family = 'Arial Black',
                                      font_color = 'black')])

fig.update_traces(textinfo = 'label + percent parent')
                  
fig.show()

# End